<a href="https://colab.research.google.com/github/sfranchois/pyspark_exercise/blob/main/pyspark_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark homework assignment

## Context

The goal of this assignment is to get view on your coding workflow & style.  Your main focus should be creating performant & robust code for data manipulations.  

For a homework assignment, we cannot grant you access to our infrastructure (Cloudera data platform on prem: a spark cluster deployment on Yarn).  Since the focus is on development, we provided a template notebook to get up and running very quickly on Google Colab.  

You have the freedom to perform this assignment on any spark3+ infrastructure.  If want to use a local or cloud setup, go for it!

Some of the tasks are open for interpretation.  This allows us to assess business understanding and relevant field experience.  These tasks are not pass or fail checks.  During the interview we'll ask details about the choice(s) you made.

For the assignment, you'll be working with store location data.  You might be familiar with the phrase "Location, location, location" from the real-estate context.  The same house can have a different selling price based on the location.  In fast moving consumer goods (FMCG), location is one of the most crucial aspects:

* Proximity & accessibility to customers increases convenience
* Proximity to competitors increases market pressure
* It has impact on the supply chain

## Evaluation criteria

1. Software engineering
   1. Clean code (e.g. using meaningful names)
   1. Robust & efficient code
   1. Styling (e.g. PEP8, or Google style guide)
   1. Documentation(e.g. docstrings)
   1. Design (e.g. SOLID principles)
1. Workflow
   1. How you use Git
   1. How you structure your assignment
   1. Owning mistakes
   1. Rationale for design decisions
   1. Making your solution accessible to others
1. Business context
   1. GDPR
   1. Fast moving consumer goods
1.(optional: own infra) System engineering
   1. What setup did you use?
   1. How did you set it up?

## Deliverables we expect

1. Private GitHub repo
   1. Colab allows you to save to GitHub
   1. Invite my username to your private repo as contributor
1. README.md with relevant content
1. Code relevant to the assignment


# **README**

**Platform**

MacOS Sonoma 14.3.1


**Steps**

* clone repo https://github.com/MarkiesFredje/pyspark-exercise
* read instructions in notebook
* brew install openjdk@11
* created virtualenv with pyenv
* pip installed jupyter findspark pyarrow pandas keyring
* start jupyter-notebook
* point env vars to my dirs
* set spark local ip env var
* implement notebook
* uploaded notebook to Google Colab space


Initially, the code was written with Pandas DataFrames.
I'm on a tight schedule.

Setting up Spark inside Colab was done in another way.
Java is already installed there, so skipped.

Migration to use RDD DataFrames ongoing.

## Google Colab spark setup

In [ ]:
# commented out: see below
#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
#!tar xzvf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark pandas pyarrow

In [ ]:
from os import environ
import findspark
import requests
import subprocess
import os
import re
import socket
import shutil
import time
import sys

def run(cmd):
    # run a shell command
    try:
        # Run the command and capture stdout and stderr
        subprocess_output = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        # Access stdout (stderr redirected to stdout)
        stdout_result = subprocess_output.stdout.strip().splitlines()[-1]
        # Process the results as needed
        print(f'✅ {stdout_result}')
        return stdout_result
    except subprocess.CalledProcessError as e:
        # Handle the error if the command returns a non-zero exit code
        print(f"Command failed with return code {e.returncode}")
        print("stdout:", e.stdout)

def is_java_installed():
    return shutil.which("java")

def install_java():
    # Uncomment and modify the desired version
    # java_version= 'openjdk-11-jre-headless'
    # java_version= 'default-jre'
    # java_version= 'openjdk-17-jre-headless'
    # java_version= 'openjdk-18-jre-headless'
    java_version= 'openjdk-19-jre-headless'
    os.environ['JAVA_HOME'] = ' /usr/lib/jvm/java-19-openjdk-amd64'
    print(f"Java not found. Installing {java_version} ... (this might take a while)")
    try:
        cmd = f"apt install -y {java_version}"
        subprocess_output = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        stdout_result = subprocess_output.stdout
        # Process the results as needed
        print(f'✅ Done installing Java {java_version}')
    except subprocess.CalledProcessError as e:
        # Handle the error if the command returns a non-zero exit code
        print(f"Command failed with return code {e.returncode}")
        print("stdout:", e.stdout)

print("\n0️⃣   Install Java if not available")
if is_java_installed():
    print("✅ Java is already installed.")
else:
    install_java()

print("\n1️⃣   Download and install Hadoop and Spark")
# URL for downloading Hadoop and Spark
SPARK_VERSION = "3.5.1"
HADOOP_SPARK_URL = "https://dlcdn.apache.org/spark/spark-" + SPARK_VERSION + \
                   "/spark-" + SPARK_VERSION + "-bin-hadoop3.tgz"
r = requests.head(HADOOP_SPARK_URL)
if r.status_code >= 200 and r.status_code < 400:
    print(f'✅ {HADOOP_SPARK_URL} was found')
else:
    SPARK_CDN = "https://dlcdn.apache.org/spark/"
    print(f'⚠️ {HADOOP_SPARK_URL} was NOT found. \nCheck for available Spark versions in {SPARK_CDN}')

# set some environment variables
os.environ['SPARK_HOME'] = os.path.join(os.getcwd(), os.path.splitext(os.path.basename(HADOOP_SPARK_URL))[0])
os.environ['PATH'] = ':'.join([os.path.join(os.environ['SPARK_HOME'], 'bin'), os.environ['PATH']])
os.environ['PATH'] = ':'.join([os.path.join(os.environ['SPARK_HOME'], 'sbin'), os.environ['PATH']])

# download Spark
# using --no-clobber option will prevent wget from downloading file if already present
# shell command: wget --no-clobber $HADOOP_SPARK_URL
cmd = f"wget --no-clobber {HADOOP_SPARK_URL}"
run(cmd)

# uncompress
try:
    # Run the command and capture stdout and stderr
    cmd = "([ -d $(basename {0}|sed 's/\.[^.]*$//') ] && echo -n 'Folder already exists') || (tar xzf $(basename {0}) && echo 'Uncompressed Spark distribution')"
    subprocess_output = subprocess.run(cmd.format(HADOOP_SPARK_URL), shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    # Access stdout (stderr redirected to stdout)
    stdout_result = subprocess_output.stdout
    # Process the results as needed
    print(f'✅ {stdout_result}')

except subprocess.CalledProcessError as e:
    # Handle the error if the command returns a non-zero exit code
    print(f"Command failed with return code {e.returncode}")
    print("stdout:", e.stdout)


print("\n2️⃣   Start Spark engine")
# start master
# shell command: $SPARK_HOME/sbin/start-master.sh
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-master.sh')
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-master.sh')
out = run(cmd)

# start one worker (first stop it in case it's already running)
# shell command: $SPARK_HOME/sbin/start-worker.sh spark://${HOSTNAME}:7077
cmd = [os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-worker.sh')]
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-worker.sh') + ' ' + 'spark://'+socket.gethostname()+':7077'
run(cmd)

print("\n3️⃣   Start Master Web UI")
# get master UI's port number
# the subprocess that's starting the master with start-master.sh
# might still not be ready with assigning the port number at this point
# therefore we check the logfile a few times (attempts=5) to see if the port
# has been assigned. This might take 1-2 seconds.

master_log = out.partition("logging to")[2].strip()
print("Search for port number in log file {}".format(master_log))
attempts = 10
search_pattern = "Successfully started service 'MasterUI' on port (\d+)"
found = False
for i in range(attempts):
  if not found:
   with open(master_log) as log:
      found = re.search(search_pattern, log.read())
      if found:
          webUIport = found.group(1)
          print(f"✅ Master UI is available at localhost:{webUIport} (attempt nr. {i})")
          break
      else:
          time.sleep(2) # need to try until port information is found in the logfile
          i+=1
if not found:
  print("Could not find port for Master Web UI\n")

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    # serve the Web UI on Colab
    print("Click on the link below to open the Spark Web UI 🚀")
    from google.colab import output
    output.serve_kernel_port_as_window(webUIport)

print("\n4️⃣   Start history server")
# start history server
# shell command: mkdir -p /tmp/spark-events
# shell command: $SPARK_HOME/sbin/start-history-server.sh
spark_events_dir = os.path.join('/tmp', 'spark-events')
if not os.path.exists(spark_events_dir):
    os.mkdir(spark_events_dir)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-history-server.sh')
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-history-server.sh')
run(cmd)

if IN_COLAB:
    # serve the History Server
    print("Click on the link below to open the Spark History Server Web UI 🚀")
    output.serve_kernel_port_as_window(18080)



0️⃣   Install Java if not available
✅ Java is already installed.

1️⃣   Download and install Hadoop and Spark
✅ https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz was found
✅ 2024-03-13 09:05:15 (71.6 MB/s) - ‘spark-3.5.1-bin-hadoop3.tgz’ saved [400446614/400446614]
✅ Uncompressed Spark distribution


2️⃣   Start Spark engine
✅ no org.apache.spark.deploy.master.Master to stop
✅ starting org.apache.spark.deploy.master.Master, logging to /content/spark-3.5.1-bin-hadoop3/logs/spark--org.apache.spark.deploy.master.Master-1-ad5e23a0f094.out
✅ no org.apache.spark.deploy.worker.Worker to stop
✅ starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-3.5.1-bin-hadoop3/logs/spark--org.apache.spark.deploy.worker.Worker-1-ad5e23a0f094.out

3️⃣   Start Master Web UI
Search for port number in log file /content/spark-3.5.1-bin-hadoop3/logs/spark--org.apache.spark.deploy.master.Master-1-ad5e23a0f094.out
✅ Master UI is available at localhost:8081 (attempt nr. 2)


<IPython.core.display.Javascript object>


4️⃣   Start history server
✅ no org.apache.spark.deploy.history.HistoryServer to stop
✅ starting org.apache.spark.deploy.history.HistoryServer, logging to /content/spark-3.5.1-bin-hadoop3/logs/spark--org.apache.spark.deploy.history.HistoryServer-1-ad5e23a0f094.out
Click on the link below to open the Spark History Server Web UI 🚀


<IPython.core.display.Javascript object>

In [ ]:
# Setting environment variables
# These settings are for MacOS and not needed in Colab
#environ["JAVA_HOME"] = "/usr/local/opt/openjdk@11"
#environ["SPARK_HOME"] = "./spark-3.4.1-bin-hadoop3"
#environ["SPARK_LOCAL_IP"] = "10.0.1.10"

environ["SPARK_HOME"]

'/content/spark-3.5.1-bin-hadoop3'

In [ ]:
# Init spark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
# spark.sql.repl.eagerEval.enabled: Property used to format output tables better

spark = (
    SparkSession
    .builder
    .appName("cg-pyspark-assignment")
    .master("local")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .getOrCreate()
  )

spark

In [ ]:
os.getcwd()

'/content'

## Getting the assignment data

This will call the api and save the results in current working directory as .json files

In [ ]:
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places > clp-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/okay-places > okay-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/spar-places > spar-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/dats-places > dats-places.json
!curl https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/cogo-colpnts > cogo-colpnts.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  222k    0  222k    0     0   146k      0 --:--:--  0:00:01 --:--:--  146k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146k    0  146k    0     0   159k      0 --:--:-- --:--:-- --:--:--  159k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  165k    0  165k    0     0   171k      0 --:--:-- --:--:-- --:--:--  171k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89162    0 89162    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
  % Total    % Received % Xferd  Average Speed   Tim

## Assignment instructions

1. Download the data from api
1. Create a logger object that logs to a file "assignment.log"
   1. You can add whatever logging config you want or need
   1. At least on Filehandler based on instructions
1. implement get_data_by_brand function
   1. Follow instructions in docstring
   1. df_clp code line should work
1. No more handholding ... :-)
1. Create a single object (dataframe) that:
   1. Contains data from **all brands**
      1. Not every brand has the same columns!
   1. Drop placeSearchOpeningHours
   1. You can keep sellingPartners as an array
   1. Extract "postal_code" from address
   1. Create new column "province" derived from postal_code
   1. Transform geoCoordinates into lat and lon column
   1. One-hot-encode the handoverServices
   1. Pretend houseNumber and streetName are GDPR sensitive.
      1. How would you anonymize this data for unauthorized users?
      1. (optional) Implement the above
      1. How would you show the real data to authorized users?
      1. (optional) Implement the above
1. Save the end result as a parquet file
   1. (optional)partitioning?

**postal_code** logic:
* "Brussel": 1000-1299  
* "Waals-Brabant": 1300-1499  
* "Vlaams-Brabant": 1500-1999, 3000-3499  
* "Antwerpen": 2000-2999  
* "Limburg": 3500-3999  
* "Luik": 4000-4999  
* "Namen": 5000-5999  
* "Henegouwen": 6000-6599,7000-7999  
* "Luxemburg": 6600-6999  
* "West-Vlaanderen": 8000-8999  
* "Oost-Vlaanderen": 9000-9999

# **Notes**

Pandas DataFrames are a different breed than Spark RDD DataFrames.
Could not fit Spark in the Google Colab workspace (Free plan) initially using the provided boilerplate.
Bootstrap used from https://colab.research.google.com/github/groda/big_data/blob/master/Run_Spark_on_Google_Colab.ipynb

Pandas used here FTTB for demonstration purposes. Porting ongoing...

references:
https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/#read-json-file

In [ ]:
from logging import basicConfig, getLogger, INFO, Logger
import dataclasses
import pandas as pd
import os
from typing import List, Dict, Optional
from cryptography.fernet import Fernet
import pyarrow as pa
from abc import ABCMeta

In [ ]:
basicConfig(
     filename='assignment.log',
     level=INFO,
     format= '[%(asctime)s] {%(pathname)s:%(lineno)d} %(levelname)s - %(message)s',
     datefmt='%H:%M:%S'
 )
LOGGER = getLogger(__name__)

LOGGER.info("working dir %s", os.getcwd())

In [ ]:
@dataclasses.dataclass
class BrandInfo:
    clp: pd.DataFrame
    okay: pd.DataFrame
    spar: pd.DataFrame
    dats: pd.DataFrame
    cogo: pd.DataFrame

    df: pd.DataFrame = dataclasses.field(init=False)

    @classmethod
    def load_from_json(cls) -> None:
        return cls(
            clp = pd.read_json("clp-places.json"),
            okay = pd.read_json("okay-places.json"),
            spar = pd.read_json("spar-places.json"),
            dats = pd.read_json("dats-places.json"),
            cogo = pd.read_json("cogo-colpnts.json"),
        )

    def __post_init__(self) -> None:
        self.validate()

        self.clp['brand'] = 'clp'
        self.okay['brand'] = 'okay'
        self.spar['brand'] = 'spar'
        self.dats['brand'] = 'dats'
        self.cogo['brand'] = 'cogo'

        self.df = pd.concat(self.aslist())


    def aslist(self) -> List[pd.DataFrame]:
        return [self.clp, self.okay,  self.spar, self.dats, self.cogo]

    def get_all_columns(self) -> List[str]:
        return set().union(
            set(self.clp.columns),
            set(self.okay.columns),
            set(self.spar.columns),
            set(self.dats.columns),
            set(self.cogo.columns)
        )

    def get_common_columns(self) -> List[str]:
            return self.get_all_columns().intersection(
                set(self.clp.columns),
                set(self.okay.columns),
                set(self.spar.columns),
                set(self.dats.columns),
                set(self.cogo.columns)
            )

    def validate(self) -> None:
        if any(map( lambda df: df.empty, self.aslist())):
            raise ValueError("one or more JSON files have no data")


def load_brand_data() -> pd.DataFrame:
    return BrandInfo.load_from_json()



In [ ]:
def get_data_by_brand(brand: str, logger: Logger = LOGGER):
    """Fetch input data based on brand.

    Please add a column to the data indicating the input brand
    Please add minimum one sanity check for loading the data
    Please log things you consider relevant

    :param brand: allowed values are (clp, okay, spar, dats, cogo)
    :param logger: Logger object for logging

    :return: The relevant dataframe
    """
    brands = load_brand_data()

    logger.info("columns found: %s", brands.get_all_columns())
    logger.info("common columns : %s", brands.get_common_columns())

    return brands.df[brands.df["brand"] == brand]


In [ ]:
try:
    pd.set_option("display.max_rows", None)
    pd.set_option("future.no_silent_downcasting", True)
except pd.errors.OptionError:
    pass

In [ ]:
df_clp = get_data_by_brand(brand="clp", logger=LOGGER)

In [ ]:
df_clp.drop(["placeSearchOpeningHours"], axis=1)

,placeId,ensign,commercialName,branchId,sourceStatus,placeType,sellingPartners,handoverServices,geoCoordinates,address,moreInfoUrl,routeUrl,isActive,temporaryClosures,brand
0,902,"{'id': 8, 'name': 'COLR_Colruyt'}",AALST (COLRUYT),4156,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.933074, 'longitude': 4.0538972}","{'streetName': 'BRUSSELSE STEENWEG', 'houseNum...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,[],clp
1,946,"{'id': 8, 'name': 'COLR_Colruyt'}",AALTER (COLRUYT),4218,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 51.0784761, 'longitude': 3.4500133}","{'streetName': 'LOSTRAAT', 'houseNumber': '66'...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,[],clp
2,950,"{'id': 8, 'name': 'COLR_Colruyt'}",AARSCHOT (COLRUYT),4222,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.9760369, 'longitude': 4.8110969}","{'streetName': 'LEUVENSESTEENWEG', 'houseNumbe...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.9760369,4.811...",True,[],clp
3,886,"{'id': 8, 'name': 'COLR_Colruyt'}",ALSEMBERG (COLRUYT),4138,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.7415212, 'longitude': 4.336719}","{'streetName': 'BRUSSELSESTEENWEG', 'houseNumb...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.7415212,4.336719",True,[],clp
4,783,"{'id': 8, 'name': 'COLR_Colruyt'}",AMAY (COLRUYT),3853,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.5599284, 'longitude': 5.3061951}","{'streetName': 'CHAUSSEE DE TONGRES', 'houseNu...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.5599284,5.306...",True,[],clp
5,650,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDENNE (COLRUYT),3596,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.4917055, 'longitude': 5.0930033}","{'streetName': 'RUE DE LA PAPETERIE', 'houseNu...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.4917055,5.093...",True,[],clp
6,669,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDERLECHT (HERBETTELN) COLR,3620,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.8439965, 'longitude': 4.3099483}","{'streetName': 'MAURICE HERBETTELAAN', 'houseN...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.8439965,4.309...",True,[],clp
7,744,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDERLECHT (VEEWEYDE) COLRUYT,3759,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.8275378372, 'longitude': 4.302...","{'streetName': 'BERGENSESTEENWEG', 'houseNumbe...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.8275378372,4....",True,[],clp
8,448,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDERLUES (COLRUYT),3074,IN,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",[QUALITY],[CSOP_ORDERABLE],"{'latitude': 50.401257, 'longitude': 4.2797751}","{'streetName': 'RUE DE LA STATION', 'houseNumb...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.401257,4.2797751",False,"[{'from': '04-03-2024', 'till': '17-03-2024'}]",clp
9,681,"{'id': 8, 'name': 'COLR_Colruyt'}",ANS (COLRUYT),3644,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.6588119, 'longitude': 5.5324966}","{'streetName': 'RUE

In [ ]:
df_clp["sellingPartners"]

0      [QUALITY, 3RDPARTY]
1      [QUALITY, 3RDPARTY]
2      [QUALITY, 3RDPARTY]
3      [QUALITY, 3RDPARTY]
4      [QUALITY, 3RDPARTY]
5      [QUALITY, 3RDPARTY]
6      [QUALITY, 3RDPARTY]
7      [QUALITY, 3RDPARTY]
8                [QUALITY]
9      [QUALITY, 3RDPARTY]
10     [QUALITY, 3RDPARTY]
11     [QUALITY, 3RDPARTY]
12     [QUALITY, 3RDPARTY]
13     [QUALITY, 3RDPARTY]
14     [QUALITY, 3RDPARTY]
15     [QUALITY, 3RDPARTY]
16     [QUALITY, 3RDPARTY]
17     [QUALITY, 3RDPARTY]
18     [QUALITY, 3RDPARTY]
19     [QUALITY, 3RDPARTY]
20     [QUALITY, 3RDPARTY]
21     [QUALITY, 3RDPARTY]
22     [QUALITY, 3RDPARTY]
23     [QUALITY, 3RDPARTY]
24     [QUALITY, 3RDPARTY]
25     [QUALITY, 3RDPARTY]
26     [QUALITY, 3RDPARTY]
27     [QUALITY, 3RDPARTY]
28     [QUALITY, 3RDPARTY]
29     [QUALITY, 3RDPARTY]
30     [QUALITY, 3RDPARTY]
31     [QUALITY, 3RDPARTY]
32     [QUALITY, 3RDPARTY]
33               [QUALITY]
34     [QUALITY, 3RDPARTY]
35     [3RDPARTY, QUALITY]
36     [QUALITY, 3RDPARTY]
3

In [ ]:
# extract the postal code in separate field
df_clp["postalcode"] = df_clp["address"].str["postalcode"].astype(int)

In [ ]:
postal_bins = [1000, 1300, 1500, 2000,  3000, 3500, 4000,  5000,  6000,  6600, 7000,   8000,  9000, 9999]
postal_code_mapping = {
    (1000,1299) : "Brussel",
    (1300,1499) : "Waals-Brabant",
    (1500,1999) : "Vlaams-Brabant",
    (3000,3499) : "Vlaams-Brabant",
    (2000,2999) : "Antwerpen",
    (3500,3999) : "Limburg",
    (4000,4999) : "Luik",
    (5000,5999) : "Namen",
    (6000,6599) : "Henegouwen",
    (7000,7999) : "Henegouwen",
    (6600,6999) : "Luxemburg",
    (8000,8999) : "West-Vlaanderen",
    (9000,9999) : "Oost-Vlaanderen",
}
# binning the postal codes
df_clp["province"] = pd.cut(df_clp['postalcode'], postal_bins ,labels=postal_code_mapping.values(), ordered=False)

In [ ]:
df_clp[["province", "postalcode"]]

,province,postalcode
0,Oost-Vlaanderen,9300
1,Oost-Vlaanderen,9880
2,Antwerpen,3200
3,Vlaams-Brabant,1652
4,Luik,4540
5,Namen,5300
6,Brussel,1070
7,Brussel,1070
8,Henegouwen,6150
9,Luik,4430


In [ ]:
df_clp[['lat', 'lon']] = df_clp['geoCoordinates'].apply(pd.Series)

In [ ]:
df_clp[['geoCoordinates','lat', 'lon']]

,geoCoordinates,lat,lon
0,"{'latitude': 50.933074, 'longitude': 4.0538972}",50.933074,4.053897
1,"{'latitude': 51.0784761, 'longitude': 3.4500133}",51.078476,3.450013
2,"{'latitude': 50.9760369, 'longitude': 4.8110969}",50.976037,4.811097
3,"{'latitude': 50.7415212, 'longitude': 4.336719}",50.741521,4.336719
4,"{'latitude': 50.5599284, 'longitude': 5.3061951}",50.559928,5.306195
5,"{'latitude': 50.4917055, 'longitude': 5.0930033}",50.491706,5.093003
6,"{'latitude': 50.8439965, 'longitude': 4.3099483}",50.843997,4.309948
7,"{'latitude': 50.8275378372, 'longitude': 4.302...",50.827538,4.302574
8,"{'latitude': 50.401257, 'longitude': 4.2797751}",50.401257,4.279775
9,"{'latitude': 50.6588119, 'longitude': 5.5324966}",50.658812,5.532497


In [ ]:
df_clp["handoverServices"]

0      [CSOP_ORDERABLE, PREPAID_PARCEL]
1      [CSOP_ORDERABLE, PREPAID_PARCEL]
2      [CSOP_ORDERABLE, PREPAID_PARCEL]
3      [CSOP_ORDERABLE, PREPAID_PARCEL]
4      [CSOP_ORDERABLE, PREPAID_PARCEL]
5      [CSOP_ORDERABLE, PREPAID_PARCEL]
6      [CSOP_ORDERABLE, PREPAID_PARCEL]
7      [CSOP_ORDERABLE, PREPAID_PARCEL]
8                      [CSOP_ORDERABLE]
9      [CSOP_ORDERABLE, PREPAID_PARCEL]
10     [CSOP_ORDERABLE, PREPAID_PARCEL]
11     [PREPAID_PARCEL, CSOP_ORDERABLE]
12     [CSOP_ORDERABLE, PREPAID_PARCEL]
13     [CSOP_ORDERABLE, PREPAID_PARCEL]
14     [CSOP_ORDERABLE, PREPAID_PARCEL]
15     [PREPAID_PARCEL, CSOP_ORDERABLE]
16     [CSOP_ORDERABLE, PREPAID_PARCEL]
17     [CSOP_ORDERABLE, PREPAID_PARCEL]
18     [CSOP_ORDERABLE, PREPAID_PARCEL]
19     [CSOP_ORDERABLE, PREPAID_PARCEL]
20     [CSOP_ORDERABLE, PREPAID_PARCEL]
21     [PREPAID_PARCEL, CSOP_ORDERABLE]
22     [CSOP_ORDERABLE, PREPAID_PARCEL]
23     [CSOP_ORDERABLE, PREPAID_PARCEL]
24     [CSOP_ORDERABLE, PREPAID_PARCEL]


In [ ]:
def split_out(record):
    if isinstance(record['handoverServices'], list):
        for tag in record['handoverServices']:
            record[tag] = True
    return record

df_clp.apply(split_out,axis=1).fillna(False).drop('handoverServices', axis=1)

,CSOP_ORDERABLE,PREPAID_PARCEL,address,branchId,brand,commercialName,ensign,geoCoordinates,isActive,lat,...,moreInfoUrl,placeId,placeSearchOpeningHours,placeType,postalcode,province,routeUrl,sellingPartners,sourceStatus,temporaryClosures
0,True,True,"{'streetName': 'BRUSSELSE STEENWEG', 'houseNum...",4156,clp,AALST (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.933074, 'longitude': 4.0538972}",True,50.933074,...,https://www.colruyt.be/nl/colruyt-openingsuren...,902,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",9300,Oost-Vlaanderen,"https://maps.apple.com/?daddr=50.933074,4.0538972","[QUALITY, 3RDPARTY]",AC,[]
1,True,True,"{'streetName': 'LOSTRAAT', 'houseNumber': '66'...",4218,clp,AALTER (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 51.0784761, 'longitude': 3.4500133}",True,51.078476,...,https://www.colruyt.be/nl/colruyt-openingsuren...,946,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",9880,Oost-Vlaanderen,"https://maps.apple.com/?daddr=51.0784761,3.450...","[QUALITY, 3RDPARTY]",AC,[]
2,True,True,"{'streetName': 'LEUVENSESTEENWEG', 'houseNumbe...",4222,clp,AARSCHOT (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.9760369, 'longitude': 4.8110969}",True,50.976037,...,https://www.colruyt.be/nl/colruyt-openingsuren...,950,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",3200,Antwerpen,"https://maps.apple.com/?daddr=50.9760369,4.811...","[QUALITY, 3RDPARTY]",AC,[]
3,True,True,"{'streetName': 'BRUSSELSESTEENWEG', 'houseNumb...",4138,clp,ALSEMBERG (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.7415212, 'longitude': 4.336719}",True,50.741521,...,https://www.colruyt.be/nl/colruyt-openingsuren...,886,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",1652,Vlaams-Brabant,"https://maps.apple.com/?daddr=50.7415212,4.336719","[QUALITY, 3RDPARTY]",AC,[]
4,True,True,"{'streetName': 'CHAUSSEE DE TONGRES', 'houseNu...",3853,clp,AMAY (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.5599284, 'longitude': 5.3061951}",True,50.559928,...,https://www.colruyt.be/nl/colruyt-openingsuren...,783,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",4540,Luik,"https://maps.apple.com/?daddr=50.5599284,5.306...","[QUALITY, 3RDPARTY]",AC,[]
5,True,True,"{'streetName': 'RUE DE LA PAPETERIE', 'houseNu...",3596,clp,ANDENNE (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.4917055, 'longitude': 5.0930033}",True,50.491706,...,https://www.colruyt.be/nl/colruyt-openingsuren...,650,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",5300,Namen,"https://maps.apple.com/?daddr=50.4917055,5.093...","[QUALITY, 3RDPARTY]",AC,[]
6,True,True,"{'streetName': 'MAURICE HERBETTELAAN', 'houseN...",3620,clp,ANDERLECHT (HERBETTELN) COLR,"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.8439965, 'longitude': 4.3099483}",True,50.843997,...,https://www.colruyt.be/nl/colruyt-openingsuren...,669,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",1070,Brussel,"https://maps.apple.com/?daddr=50.8439965,4.309...","[QUALITY, 3RDPARTY]",AC,[]
7,True,True,"{'streetName': 'BERGENSESTEENWEG', 'houseNumbe...",3759,clp,ANDERLECHT (VEEWEYDE) COLRUYT,"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitude': 50.8275378372, 'longitude': 4.302...",True,50.827538,...,https://www.colruyt.be/nl/colruyt-openingsuren...,744,"[{'date': '13-03-2024', 'opens': 830, 'closes'...","{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...",1070,Brussel,"https://maps.apple.com/?daddr=50.8275378372,4....","[QUALITY, 3RDPARTY]",AC,[]
8,True,False,"{'streetName': 'RUE DE LA STATION', 'houseNumb...",3074,clp,ANDERLUES (COLRUYT),"{'id': 8, 'name': 'COLR_Colruyt'}","{'latitu

In [ ]:
class CryptoCodec(metaclass=ABCMeta):
    def encoder(self, msg) -> Optional[bytes]:
        """"""

    def decoder(self, encoded_msg) -> Optional[bytes]:
        """"""

class FernetCodec(CryptoCodec):

    def __init__(self, key: Optional[bytes] = None) -> None:
        self._key = key or Fernet.generate_key()
        self._fernet = Fernet(self._key)

    def encoder(self, msg) -> Optional[bytes]:
        return self._fernet.encrypt(msg.encode()) if msg else None

    def decoder(self, encoded_msg) -> Optional[bytes]:
        return self._fernet.decrypt(encoded_msg).decode() if encoded_msg else None

    def save_key(self, filename: str) -> None:
        """ save the private key """
        open(filename, "wb").write(self._key)

    @staticmethod
    def load_key(filename: str) -> bytes:
        """ load the private key """
        return open(filename, "rb").read()

    @property
    def key(self) -> str:
        """ get obfuscated key """
        return self._key.decode('ascii')[:-8] + "*" * 8

    @classmethod
    def from_key(cls, filename: str) -> "FernetCodec":
        return cls(key=cls.load_key(filename))


codec = FernetCodec()
priv_key = codec._key
# only this private key allows you to decrypt addresses again: do not lose uit
LOGGER.info("key: %s", codec.key)

# for this exercise, persist priv_key to fs
# todo use keyring package to store priv_key in a secure vault (KeyChain, 1Password, ..)
codec.save_key("priv_key")

class AddressCodec:

    def __init__(self, codec: CryptoCodec) -> None:
        self.codec = codec

    def encode_address(self, addr:Dict) -> Dict:
        return {
            "houseNumber": self.codec.encoder(addr.get("houseNumber")),
            "streetName" : self.codec.encoder(addr.get("streetName")),
            "postalcode": addr.get("postalcode") ,
            "cityName": addr.get("cityName")
        }

    def decode_address(self, addr:Dict) -> Dict:
        return {
            "houseNumber": self.codec.decoder(addr.get("houseNumber")),
            "streetName" : self.codec.decoder(addr.get("streetName")),
            "postalcode": addr.get("postalcode") ,
            "cityName": addr.get("cityName")
        }

In [ ]:
addr_codec = AddressCodec(codec)
df_gdpr = df_clp.assign(gdpr_address=lambda df: df.address.apply(addr_codec.encode_address)).drop(['address'], axis=1)

In [ ]:
LOGGER.info("writing brands parquet file")
df_gdpr.to_parquet("brand_data.parquet")

At this point, the GDPR-sensitive FMCG-data is persisted and the key is secured

In [ ]:
# read data back
LOGGER.info("reading brands parquet file")
df = pd.read_parquet("brand_data.parquet")

In [ ]:
# read key back from fs (or vault) and re-init codec with the private key
addr_codec = AddressCodec(FernetCodec.from_key("priv_key"))

# finally, decode the GDPR data to make it human-readable
df.assign(address=lambda df: df.gdpr_address.apply(addr_codec.decode_address)).drop(['gdpr_address'], axis=1)

,placeId,ensign,commercialName,branchId,sourceStatus,placeType,sellingPartners,handoverServices,geoCoordinates,moreInfoUrl,routeUrl,isActive,placeSearchOpeningHours,temporaryClosures,brand,postalcode,province,lat,lon,address
0,902,"{'id': 8, 'name': 'COLR_Colruyt'}",AALST (COLRUYT),4156,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.933074, 'longitude': 4.0538972}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,9300,Oost-Vlaanderen,50.933074,4.053897,"{'houseNumber': '41', 'streetName': 'BRUSSELSE..."
1,946,"{'id': 8, 'name': 'COLR_Colruyt'}",AALTER (COLRUYT),4218,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 51.0784761, 'longitude': 3.4500133}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,9880,Oost-Vlaanderen,51.078476,3.450013,"{'houseNumber': '66', 'streetName': 'LOSTRAAT'..."
2,950,"{'id': 8, 'name': 'COLR_Colruyt'}",AARSCHOT (COLRUYT),4222,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.9760369, 'longitude': 4.8110969}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.9760369,4.811...",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,3200,Antwerpen,50.976037,4.811097,"{'houseNumber': '241', 'streetName': 'LEUVENSE..."
3,886,"{'id': 8, 'name': 'COLR_Colruyt'}",ALSEMBERG (COLRUYT),4138,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.7415212, 'longitude': 4.336719}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.7415212,4.336719",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,1652,Vlaams-Brabant,50.741521,4.336719,"{'houseNumber': '19', 'streetName': 'BRUSSELSE..."
4,783,"{'id': 8, 'name': 'COLR_Colruyt'}",AMAY (COLRUYT),3853,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.5599284, 'longitude': 5.3061951}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.5599284,5.306...",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,4540,Luik,50.559928,5.306195,"{'houseNumber': '247', 'streetName': 'CHAUSSEE..."
5,650,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDENNE (COLRUYT),3596,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.4917055, 'longitude': 5.0930033}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.4917055,5.093...",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,5300,Namen,50.491706,5.093003,"{'houseNumber': '4', 'streetName': 'RUE DE LA ..."
6,669,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDERLECHT (HERBETTELN) COLR,3620,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.8439965, 'longitude': 4.3099483}",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.8439965,4.309...",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],clp,1070,Brussel,50.843997,4.309948,"{'houseNumber': '57', 'streetName': 'MAURICE H..."
7,744,"{'id': 8, 'name': 'COLR_Colruyt'}",ANDERLECHT (VEEWEYDE) COLRUYT,3759,AC,"{'id': 1, 'longName': 'Winkel', 'placeTypeDesc...","[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]","{'latitude': 50.8275378372, 'longitude': 4.302...",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.8275378372,4....",True,"[{'closes': 2000, 'date': '13-03-2024', 'isOpe...",[],

In [ ]:
# partitioning is supported in a Spark env, not in Pandas
# pa.parquet.write_to_dataset(table="brands", root_path="/" , partitioning=["branchid", ])

End of notebook.